# Last API with pylast

[Oficial Website](https://www.last.fm/api/)

[PyLast Repo](https://github.com/pylast)

In [25]:
import pandas as pd 
import numpy as np 
import pylast
import os
import json

In [2]:
API_KEY = input()
API_SECRET = input()

network = pylast.LastFMNetwork(api_key=API_KEY, api_secret=API_SECRET)

### Getting users by network

In [3]:
def get_friends(user, limit = 50): 
    try: 
        friends = user.get_friends(limit = limit)
    except pylast.WSError: 
        return []
    name_friends = [link.name for link in friends]
    return name_friends 

def build_network(network, user, network_list = [], level = 5): 
    friends = get_friends(user)
    write_in_file(friends)
    network_list.extend(friends)
    if level > 0:
        for friend in friends: 
            build_network(network, network.get_user(friend), network_list, level - 1)
    else:
        return

def write_in_file(lst: list):
    with open('users.txt', 'a') as f: 
        for i in lst:
            f.write(i)
            f.write('\n')

#user = network.get_user(username = 'allanxguimaraes')
#write_in_file(['allanxguimaraes'])
#users = [] 
#build_network(network, user, users, level = 5)

## Get user information 

Some auxiliary functions

In [102]:
def get_item_id(item: str, artist: str, istrack: bool):

    if istrack: filename = 'id_tracks.json'
    else: filename = 'album_tracks.json' 

    with open(filename, 'r+') as f:
        data = json.load(f)
        try:
            item_id = data[item + ';;' + artist]
        except KeyError: 
            item_id = len(data.keys())
            data.update({item + ';;' + artist: item_id})
            f.seek(0)
            json.dump(data, f)
        return item_id

def get_item_by_id(item_id: int, istrack: bool): 

    if istrack: filename = 'id_tracks.json'
    else: filename = 'album_tracks.json' 

    with open(filename, 'r') as f:
        data = json.load(f)
    for t in data.items():
        if t[1] == item_id:
            return t[0].split(';;') 

In [103]:
def get_user_info(network, username: str) -> dict: 

    user = network.get_user(username = username)
    info = {}

    info['name'] = username
    try: 
        info['country'] = user.get_country().name
    except AttributeError: 
        info['country'] = None
        print(username)
    info['subscriber'] = int(user.is_subscriber())
    info['playcount'] = user.get_playcount()
    info['registered_since'] = user.get_registered()

    info['loved_tracks'] = []
    for loved in user.get_loved_tracks():
        track_id = get_item_id(loved.track.title, loved.track.artist.name, True)
        info['loved_tracks'].append(track_id)

    info['latest_tracks'] = []
    for latest in user.get_recent_tracks(limit = 50):
        track_id = get_item_id(latest.track.title, latest.track.artist.name, True)
        info['latest_tracks'].append(track_id)

    info['top_albums'] = []
    for album in user.get_top_albums():
        album_id = get_item_id(album.item.title, album.item.artist.name, False)
        weight = album.weight
        info['top_albums'].append((album_id, weight))

    info['top_tracks'] = []
    for track in user.get_top_tracks():
        track_id = get_item_id(track.item.title, track.item.artist.name, True)
        weight = track.weight
        info['top_tracks'].append((track_id, weight))
    
    return info 

In [117]:
with open('users.txt', 'r') as f:
    text = f.read()
    users = text.split(sep = '\n')

users = np.unique(users)
chosen_users = np.random.choice(users, size = 30)

users_info = []
for username in chosen_users:
    info = get_user_info(network, username)
    users_info.append(info)

Now we have a small part of tha data (100 random users from the preview 44 thousand users). We've collected only some of the possible information

In [118]:
if not os.path.exists('users_info.json'):
    with open('users_info.json', 'w') as f:
        json.dump([], f)

with open('users_info.json', 'r+') as f: 
    data = json.load(f)
    data.extend(users_info)
    f.seek(0)
    json.dump(data, f)